In [1]:
from bs4 import BeautifulSoup
import sys, urllib2, time
import re

In [2]:
from bs4 import BeautifulSoup
import sys, urllib2, time
subgroup = ''
def parseExamples(text, chno):
    pgNo = 'xxx'
    import re

    def cleanHTML(raw_html):
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, '', raw_html)
        return cleantext
    
    def markupVerb(str1):
        #<span class="example">Dressing for success doesn't mean <span class="bold"><span class="underline">dolling</span> yourself <span class="bold">up</span></span> in suits and shoulder pads.</span>
        #<span class="example"><span class="bold">`You really are serious about this, aren't you?'</span> he <span class="bold"><span class="underline">asked</span></span>, nearly incredulous.</span>
        noSeg = 0
        str1 = str1.replace('</span></li>', '')
        if str1.endswith('</span></span>'):
            str1 = str1[:-14] #.replace('.</span></span>', '.').replace('?</span></span>', '?').replace('!</span></span>', '!')
        str2 = str1.replace('<span class="bold"><span class="underline">', '[verb]',1)
        noSeg += int(str2 != str1)
        str3 = str2.replace('<span class="bold"><span class="underline">', '[verb]',1)
        noSeg += int(str3 != str2)
        str4 = str3.replace('<span class="bold"><span class="bold">', '[verb]',1)
        noSeg += int(str4 != str3)
        str5 = str4.replace('<span class="bold"><span class="bold">', '[verb]',1)
        noSeg += int(str5 != str4)
        str6 = str5.replace('<span class="bold">', '[verb]',1)
        noSeg += int(str6 != str5)
        
        str7 = str6.replace('<span class="bold">', '[verb]',1)
        str71 = str7.replace('<span class="underline">', '[verb]',1)
        noSeg += int(str71 != str7)
        str72 = str71.replace('<span class="underline">', '[verb]',1)
        noSeg += int(str72 != str71)

        if noSeg and '</span></span>' in str72:
            str8 = str72.replace('</span></span>', '[/verb]',1)
            noSeg -= 1
        else:
            str8 = str72
        
        if noSeg and '</span></span>' in str8:
            str9 = str8.replace('</span></span>', '[/verb]',1)
            noSeg -= 1
        else:
            str9 = str8

        if noSeg and '</span>' in str9:
            str10 = str9.replace('</span>', '[/verb]',1)
            noSeg -= 1
        else:
            str10 = str9
        
        if noSeg and '</span>' in str10:
            str11 = str10.replace('</span>', '[/verb]', noSeg)
            noSeg -= 1
        else:
            str11 = str10
            
        str11 = str11.replace('[verb]`', '`').replace("'[/verb]", "'")
        
        if str11.lstrip().startswith('<span class="example">[verb]') and '[/verb],' in str11:
            str11 = str11.replace('[verb]', '', 1).replace('[/verb],', ',')
        str11 = str11.replace('.[/verb]', '[/verb].').replace(',[/verb]', '[/verb],')    
        return str11

    def abbreviate(str1):
        
        mapWord = {'Complement':'Comp', 'prepositional': 'prep', 'Adjunct': 'Adjunct', \
                    'to-inf':'to-inf', 'that':'that', 'phrase': 'phr', 'phase': 'phr', \
                    'object': 'obj', 'objects': 'objs', 'Object': 'Obj', 'Objects': 'Objs', '\
                    Auxiliaries': 'Aux', 'Auxiliary': 'Aux', 'auxiliaries': 'aux', 'auxiliary': 'aux', \
                    'subject': 'subj', 'Subject': 'Subj', 'Auxiliaries': 'Aux', 'auxiliary': 'aux', \
                    'V-ed': 'V-ed', '-ing': '-ing', 'Passive': 'Pas.', 'voice': 'v.', \
                    'Pattern': 'Pat', 'pattern': 'pat', 'Patterns': 'Pat', 'patterns': 'pat', \
                    'combination': 'comb', 'combinations': 'comb', 'Introductory': 'Intro', \
                    'Combination': 'Comb', 'Combinations': 'Comb', 'Introduction': 'Intro/Erg Verb', \
                     'frequent': 'freq', 'Other': 'Oth', 'related': '', 'structures': 'str'}
        words, res = str1.split(' '), []
        for word in words:
            res += [ mapWord[word] if word in mapWord else word ]
        return ' '.join(res)
    
    htmlText = file(text).read()
    soup = BeautifulSoup(htmlText, "html.parser")
    
            
    selectedData = soup.find_all(['div','div', 'div', 'div', 'div', 'div', 'span', 'ul', 'div', 'span'], \
                            class_=['hd8','hd1', 'hd2', 'hd3', 'hd5', 'gx', 'example', 'example', 'list', 'pg'])
    key, hdLevel = {1: '', 2: '', 3: '', }, 1
    
    

    for data in selectedData:
        

        if pgNo < startFromPage:
            continue
            
        classData, classText = data.get("class"), data.text
        
        
        if(classData in [["hd1"], ["hd2"], ["hd3"], ]):
            doneKey = False

        if(classData in [["hd1"], ["hd2"], ["hd3"], ]):
            
            hdLevel = int(classData[0][-1])
            for i in range(hdLevel+1,5):
                key[i] = ''
            dataText = data.text
            if ': ' in dataText:
                if 'Verb with ' in dataText:
                    last = dataText.index('Verb with ')+10
                elif 'Verbs with ' in dataText:
                    last = dataText.index('Verbs with ')+11
                elif 'Verb in ' in dataText:
                    last = dataText.index('Verb in ')+8
                elif 'Verbs in ' in dataText:
                    last = dataText.index('Verbs in ')+9
                elif 'Other related patterns of ' in dataText:
                    last = dataText.index('Other related patterns of ')+26
                else:
                    last = dataText.rfind(' ')+1 if dataText.startswith('Str') \
                                            else dataText.find(': ')+1
                #firstPart = abbreviate(dataText[:dataText.index(': ')])
                key[hdLevel] = '%s'%(abbreviate(dataText[last:].lstrip())) 
            else:
                key[hdLevel] = abbreviate(dataText)
            key2 = "<1>"
        elif(classData in [["hd5"], ]):
            newHDlevel = int(classData[0][-1])
            if hdLevel <= newHDlevel:
                hdLevel = newHDlevel
                pat = re.compile("`(.*?)'")
                keyWord = re.findall(pat, data.text)
                
                try:
                    key2 = '<%s %s>'%(data.text[:data.text.index(' ')], '/'.join(keyWord) if keyWord else 'other')
                except:
                    key2 = '<%s OTH>'%data.text[:5]
                    
        key1 = '%s%s%s' % ('[%s]'%key[1] if key[1] else '', '/%s/'%key[2] if key[2] else '', \
                             '<%s>'%key[3] if key[3] not in key[1] else '', )
        
        if(data.get("class") == ["gx"]):
            for s in data.find_all('span'):
                if s.get("class") == ["example"]:
                    
                    str1 = ("%s\t%s\t%s\tEx\t%s" % (chno+'-'+str(pgNo), key1, key2, cleanHTML(markupVerb(str(s))).replace('[verb]', '<i>').replace('[/verb]', '</i>')))
                    
                    print str1
                    
                    
        if(classData in [["hd5"],["hd3"]]):
            global subgroup
            subgroup = ''
            
        
        
        if(classData in [["hd8"]]):
            try:
                subgroup = '::'+re.findall("`+\D+\'" ,data.text)[0].replace("'",'').replace("`",'')+'>'
            except:
                subgroup = ' '.join(data.text.split(' ')[1:])
                   
        if(data.get("class") == ["example"]):
             
            for s in data.find_all('li'):
                if subgroup!='':
                    key2 = key2.replace('>','')
                str1 = "%s\t%s\t%s\tEx\t%s" % (chno+'-'+str(pgNo), key1, key2+subgroup, \
                            cleanHTML(markupVerb(str(s))).replace('[verb]', '<i>').replace('[/verb]', '</i>'))
                print str1
        
        
        if(data.get("class") == ["pg"]):
            pgNo = int(data.text)
            continue
        
    return

In [3]:
sys.stdout = open('pg.verbs.txt', 'w') 

In [4]:
startFromPage = 0
for chno in range(1,13):
    filename = ('ch%2d.html'% chno).replace(' ', '0')
    parseExamples(filename, ('%2d'% chno).replace(' ', '0'))